In [ ]:
import pandas as pd
import glob
from typing import Tuple
import torch
from torch import nn
from torch.utils.data import Dataset
from math import log
import lightning as L
from lightning.pytorch.loggers import WandbLogger
from src.small_dataset import load_as_df

In [ ]:
df = load_as_df()
print(f"Data points: {len(df)}")

In [ ]:
class ReviewsDataset(Dataset):
    """
    A PyTorch Dataset that provides access to the reviews data.

    Features:
    - delta_t_is_null: whether the delta_t is null
    - delta_t: time since last review
    - past_reviews:
        - past_rating_is_null: whether the past rating is null
        - past_rating: the rating of the past review
        - past_delta_t_is_null: whether the past delta_t is null
        - past_delta_t: the time since the past review
    """

    def __init__(self, df: pd.DataFrame, num_past_reviews: int) -> None:
        self.df = df
        self.num_past_reviews = num_past_reviews

    def __len__(self) -> int:
        return len(self.df)

    def __getitem__(self, index: int) -> Tuple[torch.Tensor, torch.Tensor]:
        row = self.df.iloc[index]

        delta_t_is_null = int(row["delta_t"] == -1)
        delta_t = log(1 + row["delta_t"]) if not delta_t_is_null else 0
        rating = 0 if row["rating"] == 1 else 1

        past_reviews = []
        for i in range(1, self.num_past_reviews + 1):
            past_row = self.df.iloc[index - i]

            if past_row["user"] != row["user"] or past_row["card_id"] != row["card_id"]:
                past_reviews.append(
                    [
                        1,
                        0,
                        1,
                        0,
                    ]
                )
                continue

            past_rating_is_null = 0
            past_rating = 0 if past_row["rating"] == 1 else 1
            past_delta_t_is_null = int(past_row["delta_t"] == -1)
            past_delta_t = log(1 + past_row["delta_t"]) if not past_delta_t_is_null else 0

            past_reviews.append(
                [
                    past_rating_is_null,
                    past_rating,
                    past_delta_t_is_null,
                    past_delta_t,
                ]
            )

        return (
            torch.tensor([delta_t_is_null, delta_t] + sum(past_reviews, []), dtype=torch.float32),
            torch.tensor([rating], dtype=torch.float32),
        )

In [ ]:
# Hyperparameters
batch_size = 64
num_reviews = 2
epochs = 1
train_size = 0.8
test_size = 0.2

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2 + 4 * num_reviews, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model(x)


class NNMemoryModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = NeuralNetwork()
        self.loss_fn = nn.BCELoss()

    def training_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        loss = self.loss_fn(pred, y)
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=1e-3)

    def test_step(self, batch, batch_idx):
        x, y = batch
        pred = self.model(x)
        loss = self.loss_fn(pred, y)
        self.log("test_loss", loss)

# Create dataset
dataset = ReviewsDataset(df, num_reviews)
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size]
)
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)


model = NNMemoryModel()
wandb_logger = WandbLogger(project="Memory ML")
trainer = L.Trainer(max_epochs=epochs, logger=wandb_logger)
trainer.fit(model, train_dataloader)
trainer.test(model, test_dataloader)